In [1]:
!nvidia-smi

Wed Oct 16 10:53:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0              44W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

- L4 : 25.5GB<br>
- A100 : 40GB<br>

# 라이브러리 설치 및 임포트

In [2]:
#저메모리 양자화 기법을 제공하는 라이브러리
!pip install -q -U bitsandbytes

#Hugging Face의 datasets 라이브러리
!pip install datasets -U

#Hugging Face의 transformers 라이브러리(GitHub 저장소에서 직접 최신 버전으로 설치)
!pip install -q -U git+https://github.com/huggingface/transformers.git

#Hugging Face의 PEFT(Parameter-Efficient Fine-Tuning) 라이브러리(GitHub 저장소에서 직접 최신 버전으로 설치)
!pip install -q -U git+https://github.com/huggingface/peft.git

# Hugging Face의 accelerate 라이브러리(딥러닝 모델의 학습을 가속화하는 데 사용)
!pip install -q -U git+https://github.com/huggingface/accelerate.git

# Pandas 라이브러리
!pip install pandas

#Weights & Biases (wandb) 라이브러리
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 29.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing

In [25]:
from datasets import load_dataset, Dataset
import pandas as pd
import bitsandbytes as bnb
from huggingface_hub import login, notebook_login

from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          Trainer,
                          TrainingArguments,
                          DataCollatorForSeq2Seq)


from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training)

import torch


In [26]:
# 허깅페이스 허브 로그인
api_key = "huggingface_api_key"
notebook_login()

# Qwen 2.5-3B-IT 모델 파인튜닝

## 모델 및 데이터셋 로드

In [17]:
# 모델 레포지토리
model_path = "Qwen/Qwen2.5-3B-Instruct"

# 데이터셋
df = pd.read_csv("/content/drive/MyDrive/ICT_Lecture/llm_finetuning/combined_data_3500.csv")

# input 컬럼의 NaN 데이터를 빈 문자열로 치환
df['input'] = df['input'].fillna("")
df

,input,output,instruction,data_source
0,,재료로 골뱅이와 면을 이용한 맛있는 요리를 추천해드릴게요! 이름은 '겨울로 시작해서...,"냉장고에 골뱅이 통조림, 불닭 볶음면, 청양고추, 양파가 있는데, 이 재료들로 맛있...",https://www.10000recipe.com//recipe/6889024
1,,재료로 더덕을 이용한 맛있는 요리를 추천해드릴게요! 이름은 '더덕구이'입니다. 이 ...,"대파와 마늘, 그리고 진간장이 있는데, 더덕구이 만들 수 있는 레시피 좀 알려줘!",https://www.10000recipe.com//recipe/6928555
2,,갓김치로 맛있는 볶음밥을 만들어볼까요? 오늘 소개할 요리는 '저온으로 숙성한 여수 ...,"갓김치, 베이컨, 대파가 남아있는데 이 재료로 간단하게 만들 수 있는 요리 추천해줘!",https://www.10000recipe.com//recipe/6917412
3,,재료로 흑미와 견과류를 이용한 맛있는 간식을 만들어 볼까요? 이름은 '알토란에서 나...,"냉장고에 흑미랑, 견과류, 조청이 있는데 간단한 간식 레시피 추천해줄래?",https://www.10000recipe.com//recipe/6935414
4,,재료로 바게트와 양배추를 이용한 맛있는 요리를 추천해드릴게요! 이름은 '통바게트 양...,"냉장고에 바게트와 양배추가 있는데, 간단하고 맛있는 샌드위치 레시피 추천해줘!",https://www.10000recipe.com//recipe/6916514
...,...,...,...,...
3495,,반중력에 관한 책을 읽고 있는데... ... 내려놓을 수 없다,과학에 대한 농담을 들려주세요.,airoboros
3496,,\ndef findCircleNum(isConnected):\n def dfs...,도시가 'n'개 있습니다. 그 중 일부는 연결되어 있고 일부는 연결되어 있지 않습니...,leetcode_ne
3497,"솔루션으로 A, B, C 또는 D를 선택합니다.",B,역사학자: 토지당은 1935년 바네스트리아에서 유일한 전국적 승리를 거두었습니다. ...,reclor
3498,,True,"현장 실험 결과, 새로운 품종인 그린 그램은 헥타르당 12.0 퀸탈의 수확량을 기록...",theoremqa


In [18]:
df.shape

(3500, 4)

In [21]:
# 데이터셋으로 변경
dataset = Dataset.from_pandas(df)

In [22]:
# data['train'][0]
dataset[0]

{'input': '',
 'output': "재료로 골뱅이와 면을 이용한 맛있는 요리를 추천해드릴게요! 이름은 '겨울로 시작해서 여름으로 끝나는 불닭골빔면'이에요! 30분이면 만들 수 있는 간편한 요리랍니다. 매콤하고 시원한 맛이 일품이에요!\n\n재료는 3인분 기준으로 다음과 같아요.\n\n- 골뱅이 통조림 1캔 (400g)\n- 불닭볶음면 1개\n- 비빔면 1개\n- 양파 1/2개\n- 청양고추 1개\n- 오이 1/2개\n- 초고추장 1숟가락\n- 참기름 약간\n\n그럼 함께 만들어 볼까요?\n1. 오이와 양파는 채썰고, 청양고추는 송송 썰어주세요. 골뱅이는 물기를 빼고 한 입 크기로 썰어 준비해요.\n2. 큰 볼에 준비한 재료를 넣고 불닭비빔면 소스, 비빔면 소스, 초고추장, 참기름을 넣고 잘 섞어주세요.\n3. 끓는 물에 불닭볶음면 면과 비빔면 면을 삶은 후 얼음물로 헹궈주세요.\n4. 삶은 면과 골뱅이 무침을 접시에 담은 후, 불닭볶음면 후레이크를 뿌려 완성해요!\n\n정말 간단하죠? 매콤하면서도 시원한 맛이 여름에 딱 어울리는 요리랍니다! 즐거운 요리 시간 되세요! 😊",
 'instruction': '냉장고에 골뱅이 통조림, 불닭 볶음면, 청양고추, 양파가 있는데, 이 재료들로 맛있는 요리 레시피 추천해줘!',
 'data_source': 'https://www.10000recipe.com//recipe/6889024'}

In [6]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [7]:
#   - BOS(Beginning of Sentence) 토큰
bos = tokenizer.bos_token_id

#   - EOS(End of Sentence) 토큰
eos = tokenizer.eos_token_id

#   - PAD(Padding) 토큰
pad = tokenizer.pad_token_id

#   - PAD 토큰 ID를 EOS 토큰 ID로 설정(일부 모델에서는 별도의 PAD 토큰이 없어 EOS 토큰을 PAD 토큰으로 사용)
tokenizer.pad_token_id = eos

# - 패딩을 오른쪽에 추가하도록 설정(즉, 시퀀스의 끝부분에 패딩이 추가)
tokenizer.padding_side = "right"

#   - 입력 시퀀스의 최대 길이를 2048 토큰으로 설정(이보다 긴 시퀀스는 잘림)
cut_off_len = 2048

# - 검증 세트의 크기를 전체 데이터셋의 0.5%로 설정합니다.
val_size = 0.005 # 검증세트는 추후 변경 가능

## 파인튜닝을 위한 세팅

### 입력 데이터 수정

In [8]:
prompt_input = "아래는 작업을 설명하는 지시 사항과 추가적인 문맥을 제공하는 입력이 함께 제공된 것입니다. 요청을 적절하게 완료하는 응답을 작성하세요.\n\n### 지시사항:\n{instruction}\n\n### 입력:\n{input}\n\n### 응답:\n"
prompt_no_input = "아래는 작업을 설명하는 지시 사항입니다. 요청을 적절하게 완료하는 응답을 작성하세요.\n\n### 지시사항:\n{instruction}\n\n### 응답:\n"

In [23]:
# 위에서 설정한 프롬프트에 데이터셋의 데이터를 각각의 위치에 추가
train_dataset  = dataset.map(
    lambda x: {
        'text': (
            f"{prompt_input.format(instruction=x['instruction'], input=x['input'])}{x['output']}{tokenizer.eos_token}"
            if 'input' in x and x['input'].strip() else
            f"{prompt_no_input.format(instruction=x['instruction'])}{x['output']}{tokenizer.eos_token}"
            )
        }
    )

Map:   0%|          | 0/3500 [00:00<?, ? examples/s]

In [24]:
dataset[0]

{'input': '',
 'output': "재료로 골뱅이와 면을 이용한 맛있는 요리를 추천해드릴게요! 이름은 '겨울로 시작해서 여름으로 끝나는 불닭골빔면'이에요! 30분이면 만들 수 있는 간편한 요리랍니다. 매콤하고 시원한 맛이 일품이에요!\n\n재료는 3인분 기준으로 다음과 같아요.\n\n- 골뱅이 통조림 1캔 (400g)\n- 불닭볶음면 1개\n- 비빔면 1개\n- 양파 1/2개\n- 청양고추 1개\n- 오이 1/2개\n- 초고추장 1숟가락\n- 참기름 약간\n\n그럼 함께 만들어 볼까요?\n1. 오이와 양파는 채썰고, 청양고추는 송송 썰어주세요. 골뱅이는 물기를 빼고 한 입 크기로 썰어 준비해요.\n2. 큰 볼에 준비한 재료를 넣고 불닭비빔면 소스, 비빔면 소스, 초고추장, 참기름을 넣고 잘 섞어주세요.\n3. 끓는 물에 불닭볶음면 면과 비빔면 면을 삶은 후 얼음물로 헹궈주세요.\n4. 삶은 면과 골뱅이 무침을 접시에 담은 후, 불닭볶음면 후레이크를 뿌려 완성해요!\n\n정말 간단하죠? 매콤하면서도 시원한 맛이 여름에 딱 어울리는 요리랍니다! 즐거운 요리 시간 되세요! 😊",
 'instruction': '냉장고에 골뱅이 통조림, 불닭 볶음면, 청양고추, 양파가 있는데, 이 재료들로 맛있는 요리 레시피 추천해줘!',
 'data_source': 'https://www.10000recipe.com//recipe/6889024'}

### Train/Validation 데이터셋 분할

In [38]:
# Validation Set 설정
val_set_size=0.1

# 데이터셋 분할
if val_set_size > 0:
      train_val = train_dataset.train_test_split(
            test_size=val_set_size, shuffle=True, seed=42
        )
      train_data = (
            train_val["train"]
        )
      val_data = (
            train_val["test"]
        )
else:
      train_data = train_dataset
      val_data = None

In [39]:
train_data


Dataset({
    features: ['input', 'output', 'instruction', 'data_source', 'text'],
    num_rows: 3150
})

In [40]:
val_data

Dataset({
    features: ['input', 'output', 'instruction', 'data_source', 'text'],
    num_rows: 350
})

### 토크나이징

In [41]:
# 토크나이징
train_data = train_data.map(lambda samples: tokenizer(samples["text"]), batched=True)

# input_id, attention_mask, labels 추가
train_data = train_data.map(
    lambda samples: {
        'input_ids': samples['input_ids'],
        'attention_mask': samples['attention_mask'],
        'labels': samples['input_ids']  # labels 필드 추가
    },
    batched=True
)

Map:   0%|          | 0/3150 [00:00<?, ? examples/s]

Map:   0%|          | 0/3150 [00:00<?, ? examples/s]

In [42]:
train_data

Dataset({
    features: ['input', 'output', 'instruction', 'data_source', 'text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 3150
})

In [47]:
# Validation data에도 동일한 변환 적용
val_data = val_data.map(lambda samples: tokenizer(samples["text"]), batched=True)

val_data = val_data.map(
    lambda samples: {
        'input_ids': samples['input_ids'],
        'attention_mask': samples['attention_mask'],
        'labels': samples['input_ids']  # labels 필드 추가
    },
    batched=True)

val_data

Map:   0%|          | 0/350 [00:00<?, ? examples/s]

Map:   0%|          | 0/350 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output', 'instruction', 'data_source', 'text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 350
})

In [48]:
# 불필요한 컬럼 삭제
train_data=train_data.remove_columns(['input', 'output', 'instruction', 'data_source', 'text'])
val_data=val_data.remove_columns(['input', 'output', 'instruction', 'data_source', 'text'])

In [49]:
train_data

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3150
})

In [50]:
val_data

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 350
})

## 파인튜닝 세팅

In [52]:
# 양자화
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 모델을 4비트 정밀도로 로드 / 메모리 사용량을 줄이면서도 성능 손실을 최소화
    bnb_4bit_use_double_quant=True,  # 이중 양자화(double quantization) 활성화 / 추가적인 메모리 절약
    bnb_4bit_quant_type="nf4",  # 양자화 유형을 NF4(Normal Float 4)로 설정 / 4비트 양자화에 최적화된 형식
    bnb_4bit_compute_dtype=torch.bfloat16,  # 계산 시 사용할 데이터 타입을 bfloat16으로 설정 / float32와 비슷한 범위를 가지면서도 메모리를 절약
    bnb_4bit_quant_storage=torch.bfloat16,
    )

In [53]:
# 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config = quantization_config,
    torch_dtype = torch.bfloat16,
    device_map = {"" : 0} #colab이나 gpu가 하나면 이렇게 사용
    )

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [54]:
# 모델의 캐시 사용 비활성화 : 메모리 사용량 저감 및 그래디언트 체크포인팅과 호환 / 추론 속도는 저하되나 학습 시 메모리 효율성이 향상됨
model.config.use_cache = False

# 모델을 4비트 혹은 8비트 훈련에 적합하도록 준비 : QLoRA 기법 사용 시 필요한 설정 / 양자화된 모델의 안정적인 파인튜닝을 가능케 함
model = prepare_model_for_kbit_training(model)

# 그래디언트 체크포인팅 활성화 : 순전파 중 일부 중간 활성화를 저장하지 않고 역전파 중 필요 시 재계산 / 메모리 사용량을 크게 줄일 수 있으나 계산 시간이 증가할 수 있음
model.gradient_checkpointing_enable()

In [56]:
# PEFT를 통한 LoRA 적용
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha = 16,
    target_modules = ['q_proj', 'k_proj', 'v_proj','o_proj'],
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM"
)

model = get_peft_model(model, config)

In [60]:
# 파이퍼파라미터 세팅
output_dir='/content/drive/MyDrive/ICT_Lecture/llm_finetuning/(241016)qwen_it_recipies'

num_epochs = 3

micro_batch_size = 4
gradient_accumulation_steps = 4
warmup_steps = 20
learning_rate = 1e-7
group_by_length = False
optimizer = 'paged_adamw_8bit'

# adam 활용시
beta1 = 0.9
beta2 = 0.95

lr_scheduler = 'cosine'

use_wandb = True
wandb_run_name = '(241016)_Qwen2.5_3b_IT_recipies_SFT'


use_fp16 = True
use_bf_16 = False

evaluation_strategy = 'steps'
eval_steps = 100
save_steps = 100
save_strategy = 'steps'

In [61]:
# Training Argument 설정
trainer = Trainer(
    model=model,

    train_dataset=train_data,
    eval_dataset=val_data,
    args =
    TrainingArguments(
        # max_steps=2000,
        per_device_train_batch_size = micro_batch_size,
        gradient_accumulation_steps = gradient_accumulation_steps,
        per_device_eval_batch_size = micro_batch_size,
        warmup_steps = warmup_steps,
        num_train_epochs = num_epochs,
        learning_rate = learning_rate,
        adam_beta1 = beta1, # adam 활용할때 사용
        adam_beta2 = beta2, # adam 활용할때 사용
        fp16 = use_fp16,
        bf16 = use_bf_16,
        logging_steps = 1,
        optim = optimizer,
        eval_strategy = evaluation_strategy if val_size > 0 else "no",
        save_strategy="steps",  #스텝기준으로 save
        eval_steps = eval_steps if val_size > 0 else None,
        save_steps = save_steps,
        lr_scheduler_type=lr_scheduler,
        output_dir = output_dir,
        load_best_model_at_end = True if val_size > 0 else False ,
        group_by_length=group_by_length,
        report_to="wandb" if use_wandb else None,
        run_name=wandb_run_name if use_wandb else None,

        ),
    data_collator=DataCollatorForSeq2Seq(tokenizer,padding=True),
    )

In [62]:
trainer.model.print_trainable_parameters()

trainable params: 3,686,400 || all params: 3,089,625,088 || trainable%: 0.1193


In [ ]:
wandb_api_key = '28fd0e32c761d876ae772be44a5ae15c0f83d590'

## Training

In [63]:
trainer.train()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
100,1.721700,1.714850
200,1.659700,1.714122
300,1.770100,1.713423
400,1.681900,1.712980
500,1.776200,1.712775


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-pac

TrainOutput(global_step=591, training_loss=1.6975023962317788, metrics={'train_runtime': 2119.391, 'train_samples_per_second': 4.459, 'train_steps_per_second': 0.279, 'total_flos': 1.170035690901504e+17, 'train_loss': 1.6975023962317788, 'epoch': 3.0})

In [65]:
# 모델 저장
tokenizer.save_pretrained(output_dir)
trainer.save_model()

model.push_to_hub('INo0121/qwen2.5_3b_Finetunning_241016')
tokenizer.push_to_hub('INo0121/qwen2.5_3b_Finetunning_241016')

adapter_model.safetensors:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/INo0121/qwen2.5_3b_Finetunning_241016/commit/0379b9a210c95fe751fc8d96b604d44947b80909', commit_message='Upload tokenizer', commit_description='', oid='0379b9a210c95fe751fc8d96b604d44947b80909', pr_url=None, pr_revision=None, pr_num=None)

In [67]:
from huggingface_hub import ModelCard, ModelCardData

card = ModelCard.load("INo0121/qwen2.5_3b_Finetunning_241016")
card.data = ModelCardData(
    language="ko",
    license="mit",
    tags=["korean", "qwen", "finetunned"],
    dataset_tags=["kyujinpy/KOpen-platypus"],
    metrics=["perplexity", "accuracy"],
)
card.text = """
# Qwen 2.5 3B Fine-tuned Model

This model is a fine-tuned version of Qwen 2.5 3B for recipie recommandation.

## Model Description

- Fine-tuned from: Qwen/Qwen2.5-3B
- Fine-tuning task: [Instruction-tuning]
- Training data: [kyujinpy/KOpen-platypus + Recipe data]
- Evaluation results: [Add your evaluation metrics]

## Usage

```python
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("INo0121/qwen2.5_3b_Finetunning_241016")
tokenizer = AutoTokenizer.from_pretrained("INo0121/qwen2.5_3b_Finetunning_241016")

# Example usage
input_text = "Your input text here"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
```

## Limitations and Biases

[Describe any known limitations or biases of your model]

## Training Details

- Training framework: Hugging Face Transformers
- Hyperparameters: [List your key hyperparameters]
- Training hardware: [Describe the hardware used]
"""

card.push_to_hub("INo0121/qwen2.5_3b_Finetunning_241016")

CommitInfo(commit_url='https://huggingface.co/INo0121/qwen2.5_3b_Finetunning_241016/commit/284e1e3d85f7e8857521e6db8122e11e494a7d1b', commit_message='Upload README.md with huggingface_hub', commit_description='', oid='284e1e3d85f7e8857521e6db8122e11e494a7d1b', pr_url=None, pr_revision=None, pr_num=None)

## 모델 테스트

In [69]:
# 허깅페이스 허브 로그인
api_key = "huggingface_api_key"
notebook_login()

### 기본 세팅 그대로 테스트

In [83]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from peft import PeftModel, PeftConfig

# 8비트 양자화 설정
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# 모델과 토크나이저 로드
base_model = "Qwen/Qwen2.5-3B-Instruct"
adapter_model = "INo0121/qwen2.5_3b_Finetunning_241016"

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype="auto",
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.float16,  # 추가: float16 정밀도 사용
    low_cpu_mem_usage=True,  # 추가: CPU 메모리 사용 최적화
    temperature=0.1
)
model = PeftModel.from_pretrained(model, adapter_model)
tokenizer = AutoTokenizer.from_pretrained(adapter_model)



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [84]:
def generate_chat(user_prompt=str):
    prompt = user_prompt
    messages = [
        {"role": "system", "content": """당신은 주방 보조입니다.
        사용자가 일상적인 대화를 할 경우 친절하게 응대합니다.
        사용자가 음식 추천을 요청할 경우 요청에 따라 적절한 음식을 추천하고 레시피를 자세히 알려주세요."""},
        {"role": "user", "content": prompt}
        ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1024
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [85]:
print(generate_chat("안녕?"))

안녕하세요! 어떻게 도와드릴까요? 주방에 대해 물어보실 건 있으신가요? 아니면 식사 계획을 세우고 싶으신가요?


In [87]:
print(generate_chat("소고기 채끝살이 있는데 레시피를 추천해줄래?"))

물론이죠! 소고기 채끝살은 부드럽고 담백해서 다양한 요리로 즐길 수 있습니다. 오늘은 채끝살을 활용한 간단하고 맛있는 요리를 소개하겠습니다.

### 메인 요리: 소고기 채끝살 볶음

**재료:**
- 소고기 채끝살 200g
- 양파 1개 (작게 다져서)
- 당근 1/4개 (작게 다져서)
- 생강 1스푼
- 파 2스푼
- 참기름 1스푼
- 고추장 1스푼
- 간장 1스푼
- 설탕 1스푼
- 후추 조금
- 물 2컵

**레시피:**

1. **소고기 채끝살 준비**: 
   - 채끝살은 큰 스톨에서 잘라내고, 약간의 후추와 간장을 바르고 약 15분간 고유 있게 끓여줍니다. 

2. **양파, 당근 조미료 준비**:
   - 양파와 당근은 모두 작은 조각으로 썰어줍니다.
   - 이때 생강을 함께 넣어 색상을 더해주고, 파를 넣어 향을 더합니다.

3. **볶음 준비**:
   - 팬에 기름을 두르고, 채끝살을 먼저 볶습니다. 약간의 물을 넣어 볶으면 채끝살이 부드러워집니다.
   - 양파, 당근, 생강, 파를 추가하여 볶습니다. 약간의 물을 더해 볶는 것을 잊지 마세요. 이때 참기름과 고추장을 넣어 조미료를 넣어줍니다.

4. **마무리**:
   - 마지막으로 간장을, 설탕, 그리고 조금의 후추를 넣어 조리합니다. 약 10분 정도 더 볶아주세요.
   - 정성껏 볶아서 부드러운 채끝살과 함께 맛있게 먹으실 수 있습니다.

이 요리는 간단하고 간편하게 만들 수 있는 소고기 채끝살 볶음 요리입니다. 다른 재료나 조리 방법에 대해 궁금하시면 언제든지 말씀해주세요!


| 위의 레시피 인퍼런스에 3분 소요

### 메모리 절약 관련 세팅 없이 테스트

In [88]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from peft import PeftModel, PeftConfig

# 모델과 토크나이저 로드
base_model = "Qwen/Qwen2.5-3B-Instruct"
adapter_model = "INo0121/qwen2.5_3b_Finetunning_241016"

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype="auto",
    device_map="auto",
    temperature=0.1
)
model = PeftModel.from_pretrained(model, adapter_model)
tokenizer = AutoTokenizer.from_pretrained(adapter_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [90]:
def generate_chat(user_prompt=str):
    prompt = user_prompt
    messages = [
        {"role": "system", "content": """당신은 주방 보조입니다.
        사용자가 일상적인 대화를 할 경우 친절하게 응대합니다.
        사용자가 음식 추천을 요청할 경우 요청에 따라 적절한 음식을 추천하고 레시피를 자세히 알려주세요."""},
        {"role": "user", "content": prompt}
        ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1024
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [91]:
print(generate_chat("안녕?"))

안녕하세요! 어떻게 도와드릴까요?


In [92]:
print(generate_chat("소고기 채끝살이 있는데 레시피를 추천해줄래?"))

물론이죠! 소고기 채끝살은 부드럽고 부드러운 맛이 특징으로, 다양한 요리로 활용할 수 있습니다. 오늘 소개할 레시피는 '양념된 소고기 채끝살 샐러드'입니다. 이 요리는 간단하면서도 풍미가 좋으니, 집에서 쉽게 즐길 수 있어요.

### 재료:
- 소고기 채끝살 200g
- 양배추 1/4개
- 당근 1/4개
- 생강 1/2개
- 후춧가루 약간
- 소금, 참기름, 간장 약간

### 준비 과정:
1. **소고기 채끝살 조리**:
   - 소고기 채끝살은 사전에 식초와 설탕을 바르고 30분간 숙성시킵니다. 
   - 냅킨으로 가볍게 담가주면 더욱 부드러워집니다.

2. **양배추와 당근 조리**:
   - 양배추와 당근은 송水(물)에 담가서 깨끗이 씻어줍니다.
   - 양배추는 가로로 잘라서 세로로 썰어줍니다. 당근은 반으로 잘라서 직사각형으로 썰어줍니다.

3. **요리 준비**:
   - 소고기 채끝살은 뜨거운 물에 살짝 구워서 익혀줍니다.
   - 생강은 반으로 잘라서 양배추 위에 올려놓습니다.
   - 양배추 위에 소고기 채끝살을 올려줍니다.
   - 양배추 위에 당근을 추가합니다.

4. **양념**:
   - 소금과 참기름, 간장을 양배추 위에 얹어줍니다.
   - 후춧가루를 양배추 위에 조금 더해줍니다.

5. **냉장 보관**:
   - 완성된 샐러드는 냉장보관하여 2~3일간 즐길 수 있습니다.

이렇게 간단하게 만들어진 소고기 채끝살 샐러드는 간편하면서도 고소한 맛이 일품입니다. 맛있게 드셔보세요!


| 모델을 로드할 때 메모리 절약 관련 세팅을 하지 않은 채로 로드할 경우 기존 Instruct 모델과 같이 30~40초 수준에서 추론이 수행됨

# Qwen 2.5-3B 베이스 모델 인스트럭션 튜닝